In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from pandas import read_csv
from keras.models import load_model

import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import collections
%matplotlib inline

In [ ]:
# Set up a 'look back' dataset for sequence to label prediction with Keras.

# The LSTM network expects the input data (X) to be provided with a specific
# array structure in the form of: [samples, time steps, features].

# create_dataset is adapted from
# http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

def create_dataset(X, Y, **options):
    """Convert an array of X, Y values into a dataset matrix for and LSTM"""
    
    look_back = options.pop('look_back', None)
    dataX, dataY = [], []
    for i in range(len(X) - look_back):
        a = X[i:(i+look_back)]
        dataX.append(a)
        dataY.append(Y[i + look_back - 1])
    return np.array(dataX), np.array(dataY)

# Predictions will be based on look_back minutes of data:
look_back = 50

In [ ]:
model = load_model('LSTM_Flow_Volvo.h5')

In [ ]:
path = glob.glob('C:\\Users\\XWANG221\\Downloads\\Test_2019w26_CPUload_Kiel_Amsterdam\\*.csv')

XPP_volvo = np.empty((1, 50, 22))
YPP_volvo = np.empty((1,))

for j in range(len(path)):
    
    volvo = pd.read_csv(path[j], usecols=['Lat','YawRate','Acceleration','PositionInLane','InReverse','Id1','Id2','Id6','Id7','PosLgt1','PosLgt6','PosLgt7','VelLgt6','VelLgt7','PosLat1','PosLat6','PosLat7','Type1','Type7','Lane1','Lane2','Flow'])
    if volvo.shape[0] <= 50:
        pass
    else:
        volvo['PositionInLaneRight'] = 2.5 - volvo['PositionInLane']
        volvo.fillna(10000000, inplace=True)
    
        Yvolvo = np.array(volvo['Flow'].values)
        Xvolvo = np.array(volvo[['Lat','YawRate','Acceleration','PositionInLane','PositionInLaneRight','InReverse','Id1','Id2','Id6','Id7','PosLgt1','PosLgt6','PosLgt7','VelLgt6','VelLgt7','PosLat1','PosLat6','PosLat7','Type1','Type7','Lane1','Lane2']])

        scaler = StandardScaler()
        X_volvo = scaler.fit_transform(Xvolvo)
        XP_volvo, YP_volvo = create_dataset(X_volvo, Yvolvo, look_back=look_back)
        #print(XP_volvo.shape, YP_volvo.shape)
    
        XPP_volvo = np.append(XPP_volvo, XP_volvo, axis=0)
        YPP_volvo = np.append(YPP_volvo, YP_volvo, axis=0)


In [ ]:
print(len(YPP_volvo)//64*64+1)

In [ ]:
XPP_volvo = XPP_volvo[1:14977]
YPP_volvo = YPP_volvo[1:14977]

In [ ]:
Y_predict_volvo = model.predict_classes(XPP_volvo, verbose=True)

a6 = Y_predict_volvo.tolist()
a26 = [item[0] for item in a6]
b6 = YPP_volvo.tolist()

In [ ]:
# Test_2019w26_CPUload_Kiel_Amsterdam-0011
print("Predict:", Predict, "-", "True:", Y)   
print("big:",big, "-", "precision:", precision(big), "-", "recall:", recall(big))  
print("small:",small, "-", "precision:", precision(small), "-", "recall:", recall(small)) 
print("-")
print("same:",same, "-", "precision:", precision(same), "-", "recall:", recall(same)) 
print("con95:",con95, "-", "precision:", precision(con95), "-", "recall:", recall(con95))  
print("con90:",con90, "-", "precision:", precision(con90), "-", "recall:", recall(con90)) 
print("con85:",con85, "-", "precision:", precision(con85), "-", "recall:", recall(con85)) 
print("con80:",con80, "-", "precision:", precision(con80), "-", "recall:", recall(con80))